In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
calendar = pd.read_csv('./raw_data/calendar.csv')

In [3]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [4]:
dict_day_to_group = {
    1:1, 2:1, 3:1, 4:1, 5:1,
    6:2, 7:2, 8:2, 9:2, 10:2,
    11:3, 12:3, 13:3, 14:3, 15:3,
    16:4, 17:4, 18:4, 19:4, 20:4,
    21:5, 22:5, 23:5, 24:5, 25:5,
    26:6, 27:6, 28:6, 29:6, 30:6, 31:6  
}

calendar['group_day'] = calendar['date'].str[-2:].apply(int).map(dict_day_to_group)
calendar = calendar.drop(['date','weekday'], axis=1)
calendar = calendar.rename(columns={'d':'day'})

In [5]:
calendar = pd.concat([calendar, pd.get_dummies(calendar[['event_name_1','event_name_2']])], axis=1)
calendar = calendar.drop(['event_name_1','event_name_2','event_type_1','event_type_2'], axis=1)

In [6]:
calendar

,wm_yr_wk,wday,month,year,day,snap_CA,snap_TX,snap_WI,group_day,event_name_1_Chanukah End,...,event_name_1_Ramadan starts,event_name_1_StPatricksDay,event_name_1_SuperBowl,event_name_1_Thanksgiving,event_name_1_ValentinesDay,event_name_1_VeteransDay,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster
0,11101,1,1,2011,d_1,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
1,11101,2,1,2011,d_2,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
2,11101,3,1,2011,d_3,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
3,11101,4,2,2011,d_4,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,11101,5,2,2011,d_5,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,11620,5,6,2016,d_1965,0,1,1,3,0,...,0,0,0,0,0,0,0,0,0,0
1965,11620,6,6,2016,d_1966,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
1966,11620,7,6,2016,d_1967,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
1967,11621,1,6,2016,d_1968,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
for col in [c for c in calendar.columns.tolist() if 'event_name' in c]:
    days_event = np.where(calendar[col] == 1)[0].tolist()
    calendar[col] = calendar['day']

    dict_days_event = {}
    for d in days_event:
        for i in range(0, 30):
            dict_days_event['d_'+str(d-i)] = 30-i

    calendar[col] = calendar[col].map(dict_days_event).fillna(0)

In [8]:
calendar['event_sum'] = calendar[[c for c in calendar.columns.tolist() if 'event_name' in c]].sum(axis=1).tolist()

In [9]:
calendar.head(10)

,wm_yr_wk,wday,month,year,day,snap_CA,snap_TX,snap_WI,group_day,event_name_1_Chanukah End,...,event_name_1_StPatricksDay,event_name_1_SuperBowl,event_name_1_Thanksgiving,event_name_1_ValentinesDay,event_name_1_VeteransDay,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_sum
0,11101,1,1,2011,d_1,0,0,0,6,0.0,...,0.0,23.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,46.0
1,11101,2,1,2011,d_2,0,0,0,6,0.0,...,0.0,24.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,49.0
2,11101,3,1,2011,d_3,0,0,0,6,0.0,...,0.0,25.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,52.0
3,11101,4,2,2011,d_4,1,1,0,1,0.0,...,0.0,26.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,55.0
4,11101,5,2,2011,d_5,1,0,1,1,0.0,...,0.0,27.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,58.0
5,11101,6,2,2011,d_6,1,1,1,1,0.0,...,0.0,28.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,61.0
6,11101,7,2,2011,d_7,1,0,0,1,0.0,...,0.0,29.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,64.0
7,11102,1,2,2011,d_8,1,1,1,1,0.0,...,0.0,30.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,67.0
8,11102,2,2,2011,d_9,1,1,1,2,0.0,...,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,39.0
9,11102,3,2,2011,d_10,1,1,0,2,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,42.0


In [10]:
calendar.to_csv('./proc_data/processed_calendar.csv', index=False)